# Intro

In this notebook, we will be reading from a pickle file, which will contain a list of songs through the years. Each element of this list will be a year of top songs, which is a list of dicts containing date, name, and artist. This list should contain the Billboard 100 songs for every week for the last 20 years, or as many years are available for that genre.

Given this list, we will query the spotify API to generate a dataframe that contains the same information, augmented with song ID, song features, and artist genre. This dataframe will be saved as a pickle file at the end.

In [433]:
import spotipy
import spotipy.util as util

scope = ''
username = 'testname777123'
# Note: This should be filled in with an authentication token. Make sure to delete it before committing.
token = 'BQAJPxMVHIE-xyglkb03mO7cTv0XO8yPn6WcZReTGih7kNhBLwTqiXb0VsTDx9Z9_hWN0fz4xwCD7tq_EMBUsI3UuM4Dj1LhV_9ltfmnxBAFc7cVtGl77Pa7Vh39GgJa3b9g6bH4i4kx-Noyu1_KfK7t'

sp = spotipy.Spotify(auth=token)

Let's create a cache of song mappings so as to limit our API requests to spotify.

In [400]:
# Format is dict{artist : {song : data, song : data ...} ...}
memo = dict()
unique_unfound = 0
total_unfound = 0

Now let's devise a way to get a song id, given a song and an artist:

In [401]:
from unidecode import unidecode
from fuzzywuzzy import fuzz

TITLE_DIFF = 70
ARTIST_DIFF = 70

def contains_artist(goal, artists):
    # Use unidecode to turn special characters to regular, i.e. ñ --> n
    goal = unidecode(goal).lower()
    
    for artist in artists:
        art = unidecode(artist['name']).lower()
        if fuzz.partial_ratio(art, goal) >= ARTIST_DIFF:
            return True

    return False

# If the song is not in the top 20 results, ignore it.
# TOOD: how frequently does this happen? If frequently, maybe follow the pages?
def get_song(track, artist):
    global unique_unfound
    track = unidecode(track).lower()
    artist = unidecode(artist).lower()
    
    extra_search_words = {'is', 'a', 'i', 'the'}
    
    split1 = track.split()
    first = split1[0]
    if split1[0] in extra_search_words and len(split1) > 1:
        first += ' ' + split1[1]
    
    split2 = artist.split()
    second = split2[0]
    if split2[0] in extra_search_words and len(split2) > 1:
        second += ' ' + split2[1]
        
    to_search = first + ' ' + second
    query_result = sp.search(q=to_search, type='track', limit=40)
    query_result = query_result['tracks']['items']
    
    # Clean the query results
    result = []
    for elem in query_result:
        cur_name = unidecode(elem['name']).lower()
        """
        arts = []
        for a in elem['artists']:
            arts += [a['name']]
        print(elem['name'], arts, fuzz.partial_ratio(track, cur_name), contains_artist(artist, elem['artists']))
        """
        if fuzz.partial_ratio(track, cur_name) < TITLE_DIFF:
            continue
        if not contains_artist(artist, elem['artists']):
            continue

        new_song = dict()
        new_song['name'] = cur_name
        new_song['id'] = elem['id']
        new_song['artist'] = artist
        new_song['popularity'] = elem['popularity']
        
        result += [new_song]
    
    # Return the most popular song
    result.sort(key=lambda d: d['popularity'])
    
    if len(result) == 0:
        print("Could not find '%s' by '%s' in top 20 results" % (track, artist))
        unique_unfound += 1
        return None
    
    return result[-1]

Create a wrapper to do it given a list of songs:

In [402]:
import pandas as pd
import copy

CHUNK_SIZE = 50

def divide_chunks(l, n):
    result = []
    for i in range(0, len(l), n):  
        if i >= len(l):
            result += [l[i:]]
        else:
            result += [l[i:i + n]]
    return result

def del_unneeded(d):
    del d['type']
    del d['id']
    del d['uri']
    del d['track_href']
    del d['analysis_url']
    #TODO: do we want time signature? Do we want to remove duration?
    del d['time_signature']
    
    return d

def combine_fn(a, b):
    a.update(b)
    return a

#TODO: should integrate 'retry_after' when response 429, to deal with rate limiting
# Expects song_list of type [dict{week, name, artist}....]
def convert_all_songs(song_list):
    global unique_unfound
    global total_unfound
    
    unique_unfound = 0
    total_unfound = 0
    
    result_data = []
    for song in song_list:
        artist = unidecode(song['artist']).lower()
        title = unidecode(song['name']).lower()
        if artist not in memo:
            memo[artist] = dict()
        
        # Check if we've already hit the API for this song
        if title in memo[artist]:
            temp = copy.copy(memo[artist][title])
        else:
            # Note: spotipy takes care of the 429 'retry-after' response from spotify
            temp = get_song(title, artist)
            
        if temp is not None:
            temp['date'] = song['date']
            result_data += [temp]
        else:
            total_unfound += 1
        
        memo[artist][title] = temp

    # Batch the audio features into groups of 50, which is the max number you can query spotify at once
    song_list = divide_chunks(result_data, CHUNK_SIZE)
    result_features = []
    for sublist in song_list:
        sublist = list(map(lambda d: d['id'], sublist))
        # Add the audio features, remove unneeded one
        """
        Here are the features:
        dict_keys(['danceability', 'energy', 'key', 'loudness', 'mode',
                   'speechiness', 'acousticness', 'instrumentalness', 'liveness',
                   'valence', 'tempo', 'type', 'id', 'uri', 'track_href',
                   'analysis_url', 'duration_ms', 'time_signature'])"""
        features = sp.audio_features(sublist)
        features = list(map(del_unneeded, features))
        result_features += features
    
    # merge the features into the other song data
    result = list(zip(result_data, result_features))
    result = list(map(lambda a: combine_fn(a[0], a[1]), result))
    
    return pd.DataFrame(result).set_index('date')

Here is an example of the functionality:

In [403]:
songs = [
    {'name': 'the business of emotion', 'artist': "big data featuring white sea", 'date': '2000-12-23'}
]
"""
    {'name': 'Independent Women Part I', 'artist': "Destiny's Child", 'date': '2000-12-23'}, not in
    {'name': 'He Loves U Not', 'artist': 'Dream', 'date': '2000-12-30'}, not in
    {'name': 'She Misses Him', 'artist': 'Tim Rushlow', 'date': '2000-12-30'}, not in
    {'name': 'Get Crunked Up', 'artist': 'Iconz Featuring Tony Manshino', 'date': '2000-12-30'} 69
    
    {'name': 'NAStradamus', 'artist': 'Nas', 'date': '2000-12-30'}
    {'name': 'Independent Women Part I', 'artist': "Destiny's Child", 'date': '2000-12-30'},
    {'name': 'Case Of The Ex (Whatcha Gonna Do)', 'artist': 'Mya', 'date': '2000-12-23'},
    {'name': "It Wasn't Me", 'artist': 'Shaggy Featuring Ricardo "RikRok" Ducent', 'date': '2000-12-30'},
    {'name': "It Wasn't Me", 'artist': 'Shaggy Featuring Ricardo "RikRok" Ducent', 'date': '2000-12-23'},
    {'name': 'Case Of The Ex (Whatcha Gonna Do)', 'artist': 'Mya', 'date': '2000-12-23'},
    {'name': 'I Wish', 'artist': 'R. Kelly', 'date': '??daf'}
"""
#Could not find 'fiesta' by 'r. kelly featuring jay-z' in top 20 results

#x = convert_all_songs(songs)
#x = get_song('NAStradamus', 'Nas')
x = get_song(songs[0]['name'], songs[0]['artist'])
print(x)

{'name': 'the business of emotion (feat. white sea)', 'id': '2TKso10B9HcIWy7HR1oP2g', 'artist': 'big data featuring white sea', 'popularity': 39}


Cool! Now that we have this functionality, lets run it on our data of top songs to create the end dataframe.

In [434]:
import pickle

with open('rockResults', 'rb') as f:
    top_songs = pickle.load(f)

top_songs = [song for year in top_songs for song in year] 

print(len(top_songs))

27400


In [435]:
import time
start = time.time()
result = convert_all_songs(top_songs)
print(time.time() - start)
print("Unique unfound: %s, total unfound: %s" % (unique_unfound, total_unfound))

Could not find 'life can't get much better' by 'good charlotte' in top 20 results
Could not find 'f**k with myself' by 'banks' in top 20 results
Could not find 'in the dark' by '3 doors down' in top 20 results
98.93167304992676
Unique unfound: 3, total unfound: 187


In [436]:
with open('rock.df', 'wb') as f:
    pickle.dump(result, f)

In [437]:
print(len(result))

27213


In [438]:
187/27400

0.006824817518248175